Для создания аналитического дашборда, который позволяет анализировать загруженность станций в течение дня, нам нужно вычислить и визуализировать различные метрики. Мы добавим возможность выбора временных интервалов и станций, а также рассчитаем ключевые метрики, такие как загруженность станций, пропускная способность и другие.

Вот обновленный код для Streamlit-дэшборда:

In [ ]:
import streamlit as st
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from datetime import datetime
from sklearn.cluster import KMeans

# Настройка подключения к базе данных PostgreSQL
DB_HOST = 'your_db_host'
DB_NAME = 'your_db_name'
DB_USER = 'your_db_user'
DB_PASS = 'your_db_password'

# Создание подключения к базе данных
@st.cache(allow_output_mutation=True)
def get_db_connection():
    engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}')
    return engine

# Загрузка данных из базы данных
@st.cache
def load_data():
    engine = get_db_connection()
    query = "SELECT * FROM your_table_name"  # Измените на ваше название таблицы
    df = pd.read_sql(query, engine)
    return df

# Фильтрация данных по выбранным станциям и временным интервалам
def filter_data(df, stations, start_time, end_time):
    filtered_df = df[
        (df['станция входа'].isin(stations) | df['станция выхода'].isin(stations)) &
        (df['время входа'] >= start_time) &
        (df['время выхода'] <= end_time)
    ]
    return filtered_df

# Кластеризация данных
def perform_clustering(df):
    df = df.copy()
    # Преобразование времени в количество секунд с начала дня для удобства кластеризации
    df['время входа (сек)'] = df['время входа'].dt.hour * 3600 + df['время входа'].dt.minute * 60 + df['время входа'].dt.second
    df['время выхода (сек)'] = df['время выхода'].dt.hour * 3600 + df['время выхода'].dt.minute * 60 + df['время выхода'].dt.second

    # Выбор признаков для кластеризации
    features = df[['время входа (сек)', 'время выхода (сек)', 'длительность поездки']].dropna()

    # Применение KMeans для кластеризации на 3 кластера
    kmeans = KMeans(n_clusters=3, random_state=0).fit(features)
    df['кластер'] = kmeans.labels_

    return df

# Основная функция для создания дашборда
def main():
    st.title("Аналитический дашборд")

    # Загрузка данных
    df = load_data()
    df['время входа'] = pd.to_datetime(df['время входа'])
    df['время выхода'] = pd.to_datetime(df['время выхода'])
    df['длительность поездки'] = df['время выхода'] - df['время входа']

    # Интерфейс для выбора станций и временного интервала
    stations = st.multiselect('Выберите станции', df['станция входа'].unique())
    start_time = st.slider('Начальное время', min_value=datetime(2023, 6, 1, 0, 0), max_value=datetime(2023, 6, 1, 23, 59), value=datetime(2023, 6, 1, 8, 0))
    end_time = st.slider('Конечное время', min_value=datetime(2023, 6, 1, 0, 0), max_value=datetime(2023, 6, 1, 23, 59), value=datetime(2023, 6, 1, 20, 0))

    # Фильтрация данных
    filtered_df = filter_data(df, stations, start_time, end_time)

    # Вычисление метрик для выбранного интервала
    station_load = filtered_df['станция входа'].value_counts() + filtered_df['станция выхода'].value_counts()
    max_load = station_load.max()
    station_load_percent = (station_load / max_load) * 100 if max_load != 0 else 0
    average_capacity = station_load.mean()
    top_stations = station_load.nlargest(5)
    entry_count = filtered_df['станция входа'].value_counts()
    exit_count = filtered_df['станция выхода'].value_counts()
    average_trip_duration = filtered_df['длительность поездки'].mean()

    # Кластеризация данных
    clustered_df = perform_clustering(filtered_df)

    # Визуализация данных
    st.header("Основные метрики")
    st.metric("Средняя пропускная способность", f"{average_capacity:.2f}")
    st.metric("Средняя продолжительность поездки", f"{average_trip_duration}")

    st.header("Загруженность станций (%)")
    st.bar_chart(station_load_percent)

    st.header("Топ загруженных станций")
    st.bar_chart(top_stations)

    st.header("Количество входов на станции")
    st.bar_chart(entry_count)

    st.header("Количество выходов на станции")
    st.bar_chart(exit_count)

    st.header("Кластеры поездок")
    st.write("0: Оптимальные поездки, 1: Нежелательные поездки, 2: Следует воздержаться")
    st.dataframe(clustered_df[['станция входа', 'станция выхода', 'время входа', 'время выхода', 'длительность поездки', 'кластер']])

    # Анализ загруженности станций в течение дня
    st.header("Анализ загруженности станций в течение дня")
    hourly_load = filtered_df.set_index('время входа').groupby(pd.Grouper(freq='H')).size()
    st.line_chart(hourly_load)

if __name__ == "__main__":
    main()


Объяснение кода:
Подключение к базе данных: Используем SQLAlchemy для подключения к PostgreSQL.
Загрузка данных: Загружаем данные из базы данных и кэшируем их для повышения производительности.
Фильтрация данных: Фильтруем данные по выбранным станциям и временным интервалам.
Вычисление метрик: Вычисляем основные метрики, такие как загруженность станций, пропускная способность и средняя продолжительность поездки.
Кластеризация данных: Применяем алгоритм KMeans для кластеризации данных на 3 кластера. Включаем результаты кластеризации в таблицу данных.
Визуализация данных: Создаем графики и метрики, отображающие ключевые показатели и результаты кластеризации.
Анализ загруженности станций в течение дня: Добавляем анализ загруженности станций в течение дня и визуализируем его с помощью линейного графика.


Этот обновленный дашборд позволяет пользователю проводить анализ загруженности станций в течение дня, задавать временные интервалы и выбирать станции, а также визуализировать результаты кластеризации поездок на 3 кластера, что поможет в принятии решений о поездках.






